## STRATEGY 

Parameters:
	Stretch:	0.5
	LongTermMALen:	200
	ShortTermMALen:	20
		
Data:
	ATRValue:	ATR(3)
	ADXValue:	ADX(5)
	
	ShortTermMA:	MA(Close, ShortTermMALen)
	LongTermMA:	MA(Close, LongTermMALen)

	ClosingRange:	(Close - Low) / (High - Low)
	IsUptrend: 	Close > LongTermMA
	IsVolatile:	ADXValue > 30			  

	Long_limit:	    Low - (ATRValue * Stretch) // limit order price
	Long_trigger:	ClosingRange < 0.3 // close in lower 30% of bar
	Long_setup:	    LongTrigger and IsUptrend and IsLiquid and IsVolatile
	
	StopLossExit:	Close < ShortTermMA



	Exit_rule:	Close > FillPrice or StopLossExit
	Exit_price:	NextOpen

In [52]:
import vectorbt as vbt
import pandas as pd 
import numpy as np
from datetime import datetime
from kucoin_candle_spot import SpotDataFetcher
from datetime import datetime, timezone
import pandas_ta as ta
from numba import njit
import plotly.graph_objs as go


## data collection

In [53]:

symbols = ['ETH-USDT',"BTC-USDT"]
timeframe = "1day"
start_time = "2024-01-08 10:00:00"
end_time = datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S")



# Fetch data for each symbol and store in a dictionary
dataframes = {}
for symbol in symbols:
    fetcher = SpotDataFetcher(symbol, timeframe, start_time, end_time)
    df = fetcher.fetch_candles_as_df()
    df['symbol'] = symbol
    dataframes[symbol] = df

# Print the DataFrame for each symbol
for symbol, df in dataframes.items():
    print(f"\nDataFrame for {symbol}:")
    print(df)



INFO:kucoin_candle_spot.kucoin_fetch_spot:Fetching candle data...
INFO:kucoin_candle_spot.kucoin_fetch_spot:Fetch complete. Chunks: 2, Candles: 388
INFO:kucoin_candle_spot.kucoin_fetch_spot:Fetching candle data...
INFO:kucoin_candle_spot.kucoin_fetch_spot:Fetch complete. Chunks: 2, Candles: 388



DataFrame for ETH-USDT:
                              open    close     high      low  \
timestamp                                                       
2024-01-09 00:00:00+00:00  2330.71  2344.30  2373.10  2229.25   
2024-01-10 00:00:00+00:00  2344.30  2584.58  2643.06  2340.09   
2024-01-11 00:00:00+00:00  2584.58  2617.56  2689.66  2565.79   
2024-01-12 00:00:00+00:00  2617.57  2522.33  2717.89  2457.74   
2024-01-13 00:00:00+00:00  2522.55  2578.24  2590.22  2497.73   
...                            ...      ...      ...      ...   
2025-01-26 00:00:00+00:00  3318.69  3233.03  3362.10  3229.85   
2025-01-27 00:00:00+00:00  3232.71  3182.34  3253.70  3021.15   
2025-01-28 00:00:00+00:00  3182.51  3077.63  3222.76  3039.78   
2025-01-29 00:00:00+00:00  3077.67  3114.04  3183.25  3055.00   
2025-01-30 00:00:00+00:00  3114.05  3219.62  3229.45  3091.62   

                                    volume              turnover    symbol  
timestamp                                           

In [54]:
df_btc = dataframes["BTC-USDT"]
df_eth = dataframes["ETH-USDT"]
df_btc


,open,close,high,low,volume,turnover,symbol
timestamp,,,,,,,
2024-01-09 00:00:00+00:00,46949.2,46116.8,47968.0,44620.8,6930.29094155,322277615.474301622,BTC-USDT
2024-01-10 00:00:00+00:00,46112.6,46644.8,47686.3,44316.7,12564.68340175,574891723.798530994,BTC-USDT
2024-01-11 00:00:00+00:00,46649.3,46340.1,48989.0,45578.3,8154.09494671,383028670.225992766,BTC-USDT
2024-01-12 00:00:00+00:00,46340.1,42780.2,46518.9,41440.0,7114.33128022,314646785.321163763,BTC-USDT
2024-01-13 00:00:00+00:00,42785.2,42842.7,43253.5,42433.1,2933.66783188,125738945.932025586,BTC-USDT
...,...,...,...,...,...,...,...
2025-01-26 00:00:00+00:00,104741.7,102617.4,105511.9,102511.8,876.70195799,91572400.202423731,BTC-USDT
2025-01-27 00:00:00+00:00,102606.7,102083.3,103272.8,97778.8,4766.84840827,478187302.811317367,BTC-USDT
2025-01-28 00:00:00+00:00,102071.6,101336.8,103789.3,100277.7,2133.94324915,218299424.833929149,BTC-USDT


In [55]:
def custom_trading_strategy(close, high, low, stretch=0.5, 
                             long_term_ma_len=200, short_term_ma_len=20, adx_len=5, atr_len=3):
    # Calculate indicators
    atr = vbt.IndicatorFactory.from_talib('ATR').run(high, low, close, timeperiod=atr_len).real.to_numpy()
    adx = vbt.IndicatorFactory.from_talib('ADX').run(high, low, close, timeperiod=adx_len).real.to_numpy()
    
    # Moving Averages
    long_term_ma = vbt.IndicatorFactory.from_talib('EMA').run(close, timeperiod=long_term_ma_len).real.to_numpy()
    short_term_ma = vbt.IndicatorFactory.from_talib('EMA').run(close, timeperiod=short_term_ma_len).real.to_numpy()

    # Closing Range
    closing_range = (close - low) / (high - low)
    
    # Conditions
    is_uptrend = close > long_term_ma
    is_volatile = adx > 30
    
    # Long Limit
    long_limit = low - (atr * stretch)
    
    # Long Trigger
    long_trigger = closing_range < 0.3
    
    # Long Setup
    long_setup = long_trigger & is_uptrend & is_volatile
    
    # Return limit order price when conditions for long setup are met
    limit_order_price = np.where(long_setup, long_limit, np.nan)
    
    # Ensure the order of outputs matches output_names
    return limit_order_price, long_term_ma,short_term_ma, atr, adx

# Create the indicator factory
custom_strategy_indicator = vbt.IndicatorFactory(
    class_name='CustomTradingStrategy',
    short_name='custom_strategy',
    input_names=['Close', 'High', 'Low'],
    param_names=['stretch', 'long_term_ma_len','short_term_ma_len', 'adx_len', 'atr_len'],
    output_names=['limit_order_price' ,'long_term_ma','short_term_ma', 'atr', 'adx']  # Add outputs for indicators

).from_apply_func(custom_trading_strategy)




In [57]:
# Entry parameter 
stretch = 0.5
long_term_ma_len = 200
adx_len = 5
atr_len = 3

# exit parameter
short_term_ma_len = 20
df = df_btc

# Usage example (commented out)
indicator = custom_strategy_indicator.run(
    df['close'], df['high'], df['low'],
    stretch=stretch,
    long_term_ma_len=long_term_ma_len,
    short_term_ma_len=short_term_ma_len,
    adx_len=adx_len,
    atr_len=atr_len,
    param_product=True

)

limit_price_series = indicator.limit_order_price
limit_price_series = round(limit_price_series,2)
limit_price_series[limit_price_series != np.nan]

timestamp
2024-01-09 00:00:00+00:00   NaN
2024-01-10 00:00:00+00:00   NaN
2024-01-11 00:00:00+00:00   NaN
2024-01-12 00:00:00+00:00   NaN
2024-01-13 00:00:00+00:00   NaN
                             ..
2025-01-26 00:00:00+00:00   NaN
2025-01-27 00:00:00+00:00   NaN
2025-01-28 00:00:00+00:00   NaN
2025-01-29 00:00:00+00:00   NaN
2025-01-30 00:00:00+00:00   NaN
Name: (0.5, 200, 20, 5, 3), Length: 388, dtype: float64

In [58]:
import numpy as np
import vectorbt as vbt
from numba import njit
import talib
from vectorbt.portfolio.enums import SizeType, Direction

# Create arrays to store the data we want to plot
@njit
def order_func_nb(c, high, low, open_, entries, ma_short, entry_price):  # Added entry_price parameter
    close_price = c.close[c.i, c.col]
    close_minus_1bar = c.close[c.i-1, c.col]
    
    # if in position 
    if c.position_now > 0:
        if (close_minus_1bar <= ma_short[c.i-1]) or (close_price > entry_price[c.i]):
            value = vbt.portfolio.nb.order_nb(
                size=-np.inf,
                price=open_[c.i],
                size_type=SizeType.Amount,
                direction=Direction.LongOnly,
                fees=0.001,
                slippage=0.002)
            # Store exit data
            return value

    # if not in position search for position to enter
    elif (c.position_now == 0) and (c.i != 0):
        if (entries[c.i-1] > 0) and (low[c.i] < entries[c.i-1]):
            entry_price[:] = np.nan  # Reset entry price array
            entry_price[:] = entries[c.i-1]  # Update entry price array

            order = vbt.portfolio.nb.order_nb(
                size=1,
                price=entry_price[c.i],
                size_type=SizeType.Percent,
                direction=Direction.LongOnly,
                fees=0.001,
                slippage=0.002,
                allow_partial=False,
                raise_reject=True
            )
            return order

    return vbt.portfolio.enums.NoOrder

# candle data
close = df['close']
open_ = df['open'].to_numpy().flatten()
high = df['high'].to_numpy().flatten()
low = df['low'].to_numpy().flatten()

# indicator data
entries = limit_price_series.to_numpy()
ma_short = indicator.short_term_ma.to_numpy().flatten()

# Create arrays to store data
entry_price = np.full(close.shape[0], np.nan)  # Initialize entry price array

# Create portfolio with trade_data
pf = vbt.Portfolio.from_order_func(
    close,
    order_func_nb,
    high,
    low,
    open_,
    entries,
    ma_short,
    entry_price,  # Pass entry_price array to the function
    init_cash=500
)

In [ ]:
# for symbol in symbols: 
#     # candle data
#     close = df['close']
#     open_ = df['open'].to_numpy().flatten()
#     high = df['high'].to_numpy().flatten()
#     low = df['low'].to_numpy().flatten()

#     # indicator data
#     entries = limit_price_series.to_numpy()
#     ma_short = indicator.short_term_ma.to_numpy().flatten()

#     # Create arrays to store data
#     entry_price = np.full(close.shape[0], np.nan)  # Initialize entry price array

#     # Create portfolio with trade_data
#     pf = vbt.Portfolio.from_order_func(
#         close,
#         order_func_nb,
#         high,
#         low,
#         open_,
#         entries,
#         ma_short,
#         entry_price,  # Pass entry_price array to the function
#         init_cash=500
    

In [59]:
order_records = pd.DataFrame(pf.order_records,columns=['id', 'col', 'idx', 'size', 'price', 'fees', 'side'])
order_records.set_index(order_records['id'],inplace=True)
order_records.drop(columns='id',inplace=True)
order_records

,col,idx,size,price,fees,side
id,,,,,,
0,0,205,0.007868,63488.81418,0.499500,0
1,0,207,0.007868,61370.81240,0.482837,1
2,0,231,0.007772,62002.33716,0.481872,0
3,0,232,0.007772,59292.07820,0.460809,1
4,0,265,0.007087,64888.45788,0.459888,0
5,0,267,0.007087,60692.97080,0.430153,1
6,0,274,0.007017,61182.78132,0.429294,0
7,0,275,0.007017,60516.32480,0.424617,1
8,0,344,0.004090,103604.53548,0.423769,0


: 

In [32]:
trades = pf.trades.records_readable
trades
# combined_df = df.join(trades.set_index('Entry Timestamp'), how='outer')  # Join on index
# combined_df


,Exit Trade Id,Column,Size,Entry Timestamp,Avg Entry Price,Entry Fees,Exit Timestamp,Avg Exit Price,Exit Fees,PnL,Return,Direction,Status,Position Id
0,0,close,0.007868,2024-08-01 00:00:00+00:00,63488.81418,0.499500,2024-08-03 00:00:00+00:00,61370.8124,0.482837,-17.645792,-0.035327,Long,Closed,0
1,1,close,0.007772,2024-08-27 00:00:00+00:00,62002.33716,0.481872,2024-08-28 00:00:00+00:00,59292.0782,0.460809,-22.006384,-0.045668,Long,Closed,1
2,2,close,0.007087,2024-09-30 00:00:00+00:00,64888.45788,0.459888,2024-10-02 00:00:00+00:00,60692.9708,0.430153,-30.624973,-0.066592,Long,Closed,2
3,3,close,0.007017,2024-10-09 00:00:00+00:00,61182.78132,0.429294,2024-10-10 00:00:00+00:00,60516.3248,0.424617,-5.530153,-0.012882,Long,Closed,3
4,4,close,0.004090,2024-12-18 00:00:00+00:00,103604.53548,0.423769,2024-12-20 00:00:00+00:00,97261.7866,0.397825,-26.765052,-0.063160,Long,Closed,4
5,5,close,0.004213,2025-01-08 00:00:00+00:00,94248.18012,0.397031,2025-01-09 00:00:00+00:00,94858.7024,0.399603,1.775258,0.004471,Long,Closed,5


In [65]:
check_ma_200 = ta.ema(df['close'], length=200)
fig = go.Figure()

fig.add_trace(go.Candlestick(
    x=df.index,
    open=df['open'],
    high=df['high'],
    low=df['low'],
    close=df['close'],
    name='candlesticks'))

fig.add_trace(go.Scatter(x=df.index, y=indicator.long_term_ma, mode='lines',marker=dict(color='yellow',size=20), name='200 MA'))

fig.add_trace(go.Scatter(x=df.index, y=indicator.short_term_ma, mode='lines',marker=dict(color='red',size=20), name='20 MA'))

# Add annotations for trades
for i, row in trades.iterrows():
    fig.add_annotation(
        x=row['Entry Timestamp'],
        y=row['Avg Entry Price'],
        text="Entry",
        showarrow=True,
        arrowhead=2,
        ax=0,
        ay=-40
    )

for i, row in trades.iterrows():
    fig.add_annotation(
        x=row['Exit Timestamp'],
        y=row['Avg Exit Price'],
        text="Exit",
        showarrow=True,
        arrowhead=2,
        ax=0,
        ay=40
    )
# fig.add_trad(go.Scatter(x=df.index, y=entry_exit_bars['entry_time'],))
# Enable y-axis zooming
fig.update_layout(
    yaxis=dict(fixedrange=False),  # Allow y-axis zooming
    clickmode='event+select'  # Enable click events and selection events

)


In [73]:
pf.plot()

FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#1f77b4'},
              'name': 'Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': '3233efd1-020d-4f69-a137-7821094eb3de',
              'x': array([datetime.datetime(2018, 1, 9, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2018, 1, 10, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2018, 1, 11, 0, 0, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2025, 1, 26, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2025, 1, 27, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2025, 1, 28, 0, 0, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'xaxis': 'x',
              'y': array([1285.      , 1234.000001, 1136.000001, ..., 3233.03    , 3182.34    ,
               

In [67]:
import numpy as np
import vectorbt as vbt
from numba import njit
import talib
from vectorbt.portfolio.enums import SizeType, Direction

# Custom Indicator
def custom_trading_strategy(close, high, low, stretch=0.5, 
                             long_term_ma_len=200, short_term_ma_len=20, adx_len=5, atr_len=3):
    # Calculate indicators
    atr = vbt.IndicatorFactory.from_talib('ATR').run(high, low, close, timeperiod=atr_len).real.to_numpy()
    adx = vbt.IndicatorFactory.from_talib('ADX').run(high, low, close, timeperiod=adx_len).real.to_numpy()
    
    # Moving Averages
    long_term_ma = vbt.IndicatorFactory.from_talib('EMA').run(close, timeperiod=long_term_ma_len).real.to_numpy()
    short_term_ma = vbt.IndicatorFactory.from_talib('EMA').run(close, timeperiod=short_term_ma_len).real.to_numpy()

    # Closing Range
    closing_range = (close - low) / (high - low)
    
    # Conditions
    is_uptrend = close > long_term_ma
    is_volatile = adx > 30
    
    # Long Limit
    long_limit = low - (atr * stretch)
    
    # Long Trigger
    long_trigger = closing_range < 0.3
    
    # Long Setup
    long_setup = long_trigger & is_uptrend & is_volatile
    
    # Return limit order price when conditions for long setup are met
    limit_order_price = np.where(long_setup, long_limit, np.nan)
    
    # Ensure the order of outputs matches output_names
    return limit_order_price, long_term_ma, short_term_ma, atr, adx

# Create the indicator factory
custom_strategy_indicator = vbt.IndicatorFactory(
    class_name='CustomTradingStrategy',
    short_name='custom_strategy',
    input_names=['Close', 'High', 'Low'],
    param_names=['stretch', 'long_term_ma_len','short_term_ma_len', 'adx_len', 'atr_len'],
    output_names=['limit_order_price' ,'long_term_ma','short_term_ma', 'atr', 'adx']  # Add outputs for indicators
).from_apply_func(custom_trading_strategy)

# Entry parameter 
stretch = 0.5
long_term_ma_len = 200
adx_len = 5
atr_len = 3

# exit parameter
short_term_ma_len = 20

# Usage example (commented out)
indicator = custom_strategy_indicator.run(
    df['close'], df['high'], df['low'],
    stretch=stretch,
    long_term_ma_len=long_term_ma_len,
    short_term_ma_len=short_term_ma_len,
    adx_len=adx_len,
    atr_len=atr_len,
)

limit_price_series = indicator.limit_order_price
limit_price_series = round(limit_price_series, 2)
limit_price_series[limit_price_series != np.nan]

# Order Function
@njit
def order_func_nb(c, high, low, open_, entries, ma_short, entry_price):  # Added entry_price parameter
    close_price = c.close[c.i, c.col]
    close_minus_1bar = c.close[c.i-1, c.col]
    
    # if in position 
    if c.position_now > 0:
        if (close_minus_1bar <= ma_short[c.i-1]) or (close_price > entry_price[c.i]):
            value = vbt.portfolio.nb.order_nb(
                size=-np.inf,
                price=open_[c.i],
                size_type=SizeType.Amount,
                direction=Direction.LongOnly,
                fees=0.001,
                slippage=0.002)
            # Store exit data
            return value

    # if not in position search for position to enter
    elif (c.position_now == 0) and (c.i != 0):
        if (entries[c.i-1] > 0) and (low[c.i] < entries[c.i-1]):
            entry_price[:] = np.nan  # Reset entry price array
            entry_price[:] = entries[c.i-1]  # Update entry price array

            order = vbt.portfolio.nb.order_nb(
                size=1,
                price=entry_price[c.i],
                size_type=SizeType.Percent,
                direction=Direction.LongOnly,
                fees=0.001,
                slippage=0.002,
                allow_partial=False,
                raise_reject=True
            )
            return order

    return vbt.portfolio.enums.NoOrder

# Candle data
close = df['close']
open_ = df['open'].to_numpy().flatten()
high = df['high'].to_numpy().flatten()
low = df['low'].to_numpy().flatten()

# Indicator data
entries = limit_price_series.to_numpy().flatten()
ma_short = indicator.short_term_ma.to_numpy().flatten()

# Create arrays to store data
entry_price = np.full(close.shape[0], np.nan)  # Initialize entry price array

# Create portfolio with trade_data
pf = vbt.Portfolio.from_order_func(
    close,
    order_func_nb,
    high,
    low,
    open_,
    entries,
    ma_short,
    entry_price,  # Pass entry_price array to the function
    init_cash=500
)

# Analyze the portfolio
pf.stats()
pf.plot().show()